<a href="https://colab.research.google.com/github/yasmina-daoudi/Kaggle_Bike_Share/blob/master/bike_share_demand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **I. 0. Import libraries**
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lars
from sklearn.linear_model import LassoLars
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import ARDRegression
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import TheilSenRegressor
from sklearn.linear_model import HuberRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
#from catboost import CatBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt
import pandas as pd
import seaborn as sns
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#@title **I. 1. Import data**
BASE_PATH = "/content/drive/My Drive/bike_share" #@param
#@markdown Name of imported variables:
#@markdown - train (pd.DataFrame) : The table of train data
#@markdown - test (pd.DataFrame) : The table of test data
from google.colab import drive
drive.mount('/content/drive')
train = pd.read_csv(f"{BASE_PATH}/train.csv")
test = pd.read_csv(f"{BASE_PATH}/test.csv")
print(
    f"Shape of train data {train.shape} \n"
    f"Shape of test data {test.shape}"
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Shape of train data (10886, 12) 
Shape of test data (6493, 9)


In [ ]:
# train.describe()

In [ ]:
# train.head()
# add week ends 
# season and weather get dummies
# no data missing
# train.loc[train['humidity']==0]
# extract day and time 
# features to add : day or night/ windy or not / hot or not / perfect conditions 

In [ ]:
#@title **Preprocessing Functions**
#@markdown Here you can create your preprocssing functions

def clean_column_names(data):
  columns = data.columns
  data.columns = [
                        ("_".join(
      col.encode("ascii", errors="ignore")
      .decode()
      .split(" "))
      .replace(",","")
      .replace("(","")
      .replace(")","")
      .replace("%","")
  ) for col in data.columns
  ]
  return data

def weekend(df):
  df.loc[(df['holiday']==0)&(df['workingday']==0),'weekend']=1
  df['weekend'] = df['weekend'].fillna(0)
  return(df)

def split_datetime(df):
  df['datetime'] = df['datetime'].apply(lambda x : datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
  # df['date'] = df['datetime'].apply(lambda x : x.date())
  df['weekday'] = df['datetime'].apply(lambda x : x.weekday()) 
  df['hour'] = df['datetime'].apply(lambda x : x.time().hour)
  return(df)


def season_weather_day(df):
  df['season'] = df.season.replace([1, 2, 3,4],['spring', 'summer', 'fall', 'winter'])
  df = df.join(pd.get_dummies(df.season))
  df['weather'] = df.weather.replace([1, 2, 3,4],['clear', 'cloud', 'rain', 'snow'])
  df = df.join(pd.get_dummies(df.weather))
  df['weekday'] = df.weekday.replace([0,1, 2, 3,4,5,6],['monday','tuesday','wednesday','thursday','friday','saturday','sunday'])
  df = df.join(pd.get_dummies(df.weekday))
  return(df)


def good_weather(df):
  atemp_min, atemp_max, humid_min, humid_max = df.atemp.min(), df.atemp.max(), df.humidity.min(), df.humidity.max()
  df['temp_rescaled'] = df['atemp'].apply(lambda x : 10*(2*(x-atemp_min)/(atemp_max-atemp_min)-1))
  df['humid_rescaled'] = df['humidity'].apply(lambda x : -2*(2*(x-humid_min)/(humid_max-humid_min)-1))
  df['weather_grade'] = df['temp_rescaled']+ df['humid_rescaled']
  return(df)


def good_day(df):
  df['good_day'] = df['holiday']+df['weekend']+df['spring']+df['summer']-df['winter']-df['fall']
  return(df)


def drop_col(df):
  df = df.drop(['datetime','weekday','season','weather','season'], axis = 1)
  # 'temp_rescaled','humid_rescaled']
  return(df)

In [ ]:
#@title **The Function `preprocess`**
#@markdown Here you can add the created functions to the function `preprocess`

def preprocess(data, train_data):
  """This method will be deployed on data to do the preprocesing

   Args:
    data (pd.DataFrame): The data on which the preprocessing is applied
    train_data (pd.DataFrame): The train data used to train encoders

  Returns:
    pd.DataFrame: The preprocessed data
  """
  data = clean_column_names(data)
  data = weekend(data)
  data = split_datetime(data)
  data = season_weather_day(data)
  data = good_weather(data)
  data = good_day(data)
  data = drop_col(data)
  return data


def train_preprocessing(data, target=['casual','registered','count']):
  data = preprocess(data,data)
  return data.drop(target, axis=1), data[target]

In [ ]:
X, y = train_preprocessing(train)

In [ ]:
y_casual = y[['casual']]
y_registered = y[['registered']]
y_count = y[['count']]


In [ ]:
model_casual = ExtraTreesRegressor() #@param
model_casual.fit(X, y_casual)

model_registered = ExtraTreesRegressor() #@param
model_registered.fit(X, y_registered)

model_count = ExtraTreesRegressor() #@param
model_count.fit(X, y_count)
#@markdown Output model is stored in the variable `model`

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=100, n_jobs=None, oob_score=False,
                    random_state=None, verbose=0, warm_start=False)

In [ ]:
#@title **Make predictions**
#@markdown The predictions are stored in the variable `y_pred`
X_pred = preprocess(test, train)
y_pred_casual = model_casual.predict(X_pred)
y_pred_registered = model_registered.predict(X_pred)
y_pred_count = model_count.predict(X_pred)



In [ ]:
y_pred = y_pred_casual+y_pred_registered
y_final = pd.DataFrame(y_pred).apply(lambda x : round(x))
X_final = test[['datetime']]
sub = X_final.join(y_final)
sub = sub.rename(columns={0:'count'}) 

In [ ]:
#@title **Save predictions**
#@markdown Please make sure you create the directory **Predictions** in your **BASE_PATH**
time = str(datetime.now())
now = time[:16].replace(" ", "_").replace("-","_").replace(":","h")
#@markdown Enter a filename
#@markdown > If you don't choose a filename, the predictions are saved with a timestamp in the directory **Predictions** of your **BASE_PATH**.

filename = "" #@param {type:"string"}
if filename == "":
  filename = f"{BASE_PATH}/Predictions/predictions_{now}"
sub.to_csv(f"{filename}.csv",index=False)